In [ ]:
!git clone https://github.com/ansh941/MnistSimpleCNN

Cloning into 'MnistSimpleCNN'...
remote: Enumerating objects: 507, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 507 (delta 4), reused 4 (delta 0), pack-reused 487
Receiving objects: 100% (507/507), 622.69 MiB | 31.44 MiB/s, done.
Resolving deltas: 100% (31/31), done.


## datasets.py

In [ ]:
# lokasi setiap datasets yang digunakan
import torch
import numpy as np
from PIL import Image
from torchvision import transforms

class MnistDataset(torch.utils.data.Dataset):
    def __init__(self, training=True, transform=None):
        if training==True:
            f = open('MnistSimpleCNN/data/MNIST/raw/train-images-idx3-ubyte', 'rb')
            xs = np.array(np.frombuffer(f.read(), np.uint8, offset=16))
            f.close()
            f = open('MnistSimpleCNN/data/MNIST/raw/train-labels-idx1-ubyte', 'rb')
            ys = np.array(np.frombuffer(f.read(), np.uint8, offset=8))
            f.close()
        else:
            f = open('MnistSimpleCNN/data/MNIST/raw/t10k-images-idx3-ubyte', 'rb')
            xs = np.array(np.frombuffer(f.read(), np.uint8, offset=16))
            f.close()
            f = open('MnistSimpleCNN/data/MNIST/raw/t10k-labels-idx1-ubyte', 'rb')
            ys = np.array(np.frombuffer(f.read(), np.uint8, offset=8))
            f.close()
        xs = np.reshape(xs, (-1, 28, 28, 1)).astype(np.float32)
        ys = ys.astype(np.int)
        self.x_data = xs
        self.y_data = ys
        self.transform = transform

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = Image.fromarray(self.x_data[idx].reshape(28, 28))
        y = torch.tensor(np.array(self.y_data[idx]))
        if self.transform:
            x = self.transform(x)
        x = transforms.ToTensor()(np.array(x)/255)
        return x, y


## ema.py

In [ ]:
# hyper parameter

class EMA:
    def __init__(self, model, decay):
        self.decay = decay
        self.shadow = {}
        self.original = {}

        for name, param in model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def __call__(self, model, num_updates):
        decay = min(self.decay, (1.0 + num_updates) / (10.0 + num_updates))
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - decay) * param.data + decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def assign(self, model):
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.original[name] = param.data.clone()
                param.data = self.shadow[name]

    def resume(self, model):
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                param.data = self.original[name]

##transform.py

In [ ]:
# data augmentasi

import random
import torchvision.transforms.functional as V

class RandomRotation(object):
    def __init__(self, degrees, seed=1):
        self.degrees = (-degrees, degrees)
        random.seed(seed)
    
    @staticmethod
    def get_params(degrees):
        angle = random.uniform(degrees[0], degrees[1])
        return angle

    def __call__(self, img):
        angle = self.get_params(self.degrees)
        return V.rotate(img, angle, False, False, None, None)

## modelM3.py

In [ ]:
# membuat model dengan ukuran kernel 3x3

import torch
import torch.nn as nn
import torch.nn.functional as F

class ModelM3(nn.Module):
    def __init__(self):
        super(ModelM3, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, bias=False)       # output becomes 26x26
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 48, 3, bias=False)      # output becomes 24x24
        self.conv2_bn = nn.BatchNorm2d(48)
        self.conv3 = nn.Conv2d(48, 64, 3, bias=False)      # output becomes 22x22
        self.conv3_bn = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 80, 3, bias=False)      # output becomes 20x20
        self.conv4_bn = nn.BatchNorm2d(80)
        self.conv5 = nn.Conv2d(80, 96, 3, bias=False)      # output becomes 18x18
        self.conv5_bn = nn.BatchNorm2d(96)
        self.conv6 = nn.Conv2d(96, 112, 3, bias=False)     # output becomes 16x16
        self.conv6_bn = nn.BatchNorm2d(112)
        self.conv7 = nn.Conv2d(112, 128, 3, bias=False)    # output becomes 14x14
        self.conv7_bn = nn.BatchNorm2d(128)
        self.conv8 = nn.Conv2d(128, 144, 3, bias=False)    # output becomes 12x12
        self.conv8_bn = nn.BatchNorm2d(144)
        self.conv9 = nn.Conv2d(144, 160, 3, bias=False)    # output becomes 10x10
        self.conv9_bn = nn.BatchNorm2d(160)
        self.conv10 = nn.Conv2d(160, 176, 3, bias=False)   # output becomes 8x8
        self.conv10_bn = nn.BatchNorm2d(176)
        self.fc1 = nn.Linear(11264, 10, bias=False)
        self.fc1_bn = nn.BatchNorm1d(10)
    def get_logits(self, x):
        x = (x - 0.5) * 2.0
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        conv5 = F.relu(self.conv5_bn(self.conv5(conv4)))
        conv6 = F.relu(self.conv6_bn(self.conv6(conv5)))
        conv7 = F.relu(self.conv7_bn(self.conv7(conv6)))
        conv8 = F.relu(self.conv8_bn(self.conv8(conv7)))
        conv9 = F.relu(self.conv9_bn(self.conv9(conv8)))
        conv10 = F.relu(self.conv10_bn(self.conv10(conv9)))
        flat1 = torch.flatten(conv10.permute(0, 2, 3, 1), 1)
        logits = self.fc1_bn(self.fc1(flat1))
        return logits
    def forward(self, x):
        logits = self.get_logits(x)
        return F.log_softmax(logits, dim=1)

## modelM5.py

In [ ]:
# membuat model dengan ukuran kernel 5x5

import torch
import torch.nn as nn
import torch.nn.functional as F

class ModelM5(nn.Module):
    def __init__(self):
        super(ModelM5, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, bias=False)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 5, bias=False)
        self.conv2_bn = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 96, 5, bias=False)
        self.conv3_bn = nn.BatchNorm2d(96)
        self.conv4 = nn.Conv2d(96, 128, 5, bias=False)
        self.conv4_bn = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128, 160, 5, bias=False)
        self.conv5_bn = nn.BatchNorm2d(160)
        self.fc1 = nn.Linear(10240, 10, bias=False)
        self.fc1_bn = nn.BatchNorm1d(10)
    def get_logits(self, x):
        x = (x - 0.5) * 2.0
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        conv5 = F.relu(self.conv5_bn(self.conv5(conv4)))
        flat5 = torch.flatten(conv5.permute(0, 2, 3, 1), 1)
        logits = self.fc1_bn(self.fc1(flat5))
        return logits
    def forward(self, x):
        logits = self.get_logits(x)
        return F.log_softmax(logits, dim=1)

## modelM7.py

In [ ]:
# membuat model dengan ukuran kernel 7x7

import torch
import torch.nn as nn
import torch.nn.functional as F

class ModelM7(nn.Module):
    def __init__(self):
        super(ModelM7, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, 7, bias=False)    # output becomes 22x22
        self.conv1_bn = nn.BatchNorm2d(48)
        self.conv2 = nn.Conv2d(48, 96, 7, bias=False)   # output becomes 16x16
        self.conv2_bn = nn.BatchNorm2d(96)
        self.conv3 = nn.Conv2d(96, 144, 7, bias=False)  # output becomes 10x10
        self.conv3_bn = nn.BatchNorm2d(144)
        self.conv4 = nn.Conv2d(144, 192, 7, bias=False) # output becomes 4x4
        self.conv4_bn = nn.BatchNorm2d(192)
        self.fc1 = nn.Linear(3072, 10, bias=False)
        self.fc1_bn = nn.BatchNorm1d(10)
    def get_logits(self, x):
        x = (x - 0.5) * 2.0
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        flat1 = torch.flatten(conv4.permute(0, 2, 3, 1), 1)
        logits = self.fc1_bn(self.fc1(flat1))
        return logits
    def forward(self, x):
        logits = self.get_logits(x)
        return F.log_softmax(logits, dim=1)

## train

In [ ]:
# imports module
import sys
import os
import argparse
import numpy as np 
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
from PIL import Image

def run(p_seed=0, p_epochs=3, p_kernel_size=5, p_logdir="temp"):
    # random number generator seed ------------------------------------------------#
    SEED = p_seed
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    np.random.seed(SEED)

    # ukuran kernel dari model
    KERNEL_SIZE = p_kernel_size

    # jumlah epoch yang akan dilakukan
    NUM_EPOCHS = p_epochs

    # nama dan lokasi file yang dipakai
    if not os.path.exists("MnistSimpleCNN/logs/%s"%p_logdir):
        os.makedirs("MnistSimpleCNN/logs/%s"%p_logdir)
    OUTPUT_FILE = str("MnistSimpleCNN/logs/%s/log%03d.out"%(p_logdir,SEED))
    MODEL_FILE = str("MnistSimpleCNN/logs/%s/model%03d.pth"%(p_logdir,SEED))

    # enable GPU usage ------------------------------------------------------------#
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda == False:
        print("WARNING: CPU will be used for training.")
        exit(0)

    # augmentasi data
    transform = transforms.Compose([
        RandomRotation(20, seed=SEED),
        transforms.RandomAffine(0, translate=(0.2, 0.2)),
        ])

    # memuat data yang akan ditrain
    train_dataset = MnistDataset(training=True, transform=transform)
    test_dataset = MnistDataset(training=False, transform=None)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=120, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

    # pemilihan model, ada 3 model yang tersedia(3,5,7)
    if(KERNEL_SIZE == 3):
        model = ModelM3().to(device)
    elif(KERNEL_SIZE == 5):
        model = ModelM5().to(device)
    elif(KERNEL_SIZE == 7):
        model = ModelM7().to(device)

    summary(model, (1, 28, 28))

    # hyperparameter selection ----------------------------------------------------#
    ema = EMA(model, decay=0.999)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

    # delete result file ----------------------------------------------------------#
    f = open(OUTPUT_FILE, 'w')
    f.close()

    # global variables ------------------------------------------------------------#
    g_step = 0
    max_correct = 0

    # perulangan pada proses training
    for epoch in range(NUM_EPOCHS):
        #--------------------------------------------------------------------------#
        # train process                                                            #
        #--------------------------------------------------------------------------#
        model.train()
        train_loss = 0
        train_corr = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            train_pred = output.argmax(dim=1, keepdim=True)
            train_corr += train_pred.eq(target.view_as(train_pred)).sum().item()
            train_loss += F.nll_loss(output, target, reduction='sum').item()
            loss.backward()
            optimizer.step()
            g_step += 1
            ema(model, g_step)
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{:05d}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
        train_loss /= len(train_loader.dataset)
        train_accuracy = 100 * train_corr / len(train_loader.dataset)

        #--------------------------------------------------------------------------#
        # test process                                                             #
        #--------------------------------------------------------------------------#
        model.eval()
        ema.assign(model)
        test_loss = 0
        correct = 0
        total_pred = np.zeros(0)
        total_target = np.zeros(0)
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += F.nll_loss(output, target, reduction='sum').item()
                pred = output.argmax(dim=1, keepdim=True)
                total_pred = np.append(total_pred, pred.cpu().numpy())
                total_target = np.append(total_target, target.cpu().numpy())
                correct += pred.eq(target.view_as(pred)).sum().item()
            if(max_correct < correct):
                torch.save(model.state_dict(), MODEL_FILE)
                max_correct = correct
                print("Best accuracy! correct images: %5d"%correct)
        ema.resume(model)

        #--------------------------------------------------------------------------#
        # output                                                                   #
        #--------------------------------------------------------------------------#
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100 * correct / len(test_loader.dataset)
        best_test_accuracy = 100 * max_correct / len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%) (best: {:.2f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), test_accuracy, best_test_accuracy))

        f = open(OUTPUT_FILE, 'a')
        f.write(" %3d %12.6f %9.3f %12.6f %9.3f %9.3f\n"%(epoch, train_loss, train_accuracy, test_loss, test_accuracy, best_test_accuracy))
        f.close()

        
        # pembaruan pada learning rate yang akan diperbarui setiap epoch dengan menganbil nilai tertinggi
     
        lr_scheduler.step()

p_seed = int(input ("Seeds: ")) # input seeds value
p_epoch = int(input ("Epoch: ")) #input epoch value in each trial
p_trials = int(input ("Trials: ")) #input trial value
p_kernel_size = int (input ("Kernel size: ")) #input size kernel
p_gpu = int(input ("GPU: ")) #input GPU value
p_logdir = input ("Logdir: ") #input model

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=str(p_gpu)

for x in range (p_trials):
  run(p_seed + x, p_epoch, p_kernel_size, p_logdir)

Seeds: 0
Epoch: 2
Trials: 10
Kernel size: 5
GPU: 0
Logdir: modelM5
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 24, 24]             800
       BatchNorm2d-2           [-1, 32, 24, 24]              64
            Conv2d-3           [-1, 64, 20, 20]          51,200
       BatchNorm2d-4           [-1, 64, 20, 20]             128
            Conv2d-5           [-1, 96, 16, 16]         153,600
       BatchNorm2d-6           [-1, 96, 16, 16]             192
            Conv2d-7          [-1, 128, 12, 12]         307,200
       BatchNorm2d-8          [-1, 128, 12, 12]             256
            Conv2d-9            [-1, 160, 8, 8]         512,000
      BatchNorm2d-10            [-1, 160, 8, 8]             320
           Linear-11                   [-1, 10]         102,400
      BatchNorm1d-12                   [-1, 10]              20
Total params: 1,128,180
Trainable pa

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:992: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Train Epoch: 0 [00000/60000 (0%)]	Loss: 2.835055
Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.672545
Train Epoch: 0 [24000/60000 (40%)]	Loss: 0.446485
Train Epoch: 0 [36000/60000 (60%)]	Loss: 0.381814
Train Epoch: 0 [48000/60000 (80%)]	Loss: 0.333888
Best accuracy! correct images:  9892

Test set: Average loss: 0.1459, Accuracy: 9892/10000 (98.92%) (best: 98.92%)

Train Epoch: 1 [00000/60000 (0%)]	Loss: 0.322609
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.246928
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.257343
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.147037
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.212519

Test set: Average loss: 0.1495, Accuracy: 9817/10000 (98.17%) (best: 98.92%)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 24, 24]             800
       BatchNorm2d-2           [-1, 32, 24, 24]              64
            Conv2d-3           [-1, 64, 20, 20]   

## test

In [ ]:
# import module
import sys
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import argparse
import numpy as np 
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from PIL import Image

def run(p_seed=0, p_kernel_size=5, p_logdir="temp"):

    # enable GPU usage ------------------------------------------------------------#
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda == False:
        print("WARNING: CPU will be used for training.")
        exit(0)

    # memuat data yang telah di train sebelumnya
    test_dataset = MnistDataset(training=False, transform=None)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

    # pemilihan model yang akan digunakan
    if(p_kernel_size == 3):
        model1 = ModelM3().to(device)
    elif(p_kernel_size == 5):
        model1 = ModelM5().to(device)
    elif(p_kernel_size == 7):
        model1 = ModelM7().to(device)

    model1.load_state_dict(torch.load("MnistSimpleCNN/logs/%s/model%03d.pth"%(p_logdir,p_seed)))

    model1.eval()
    test_loss = 0
    correct = 0
    wrong_images = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model1(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            wrong_images.extend(np.nonzero(~pred.eq(target.view_as(pred)).cpu().numpy())[0]+(100*batch_idx))

    np.savetxt("MnistSimpleCNN/logs/%s/wrong%03d.txt"%(p_logdir,p_seed), wrong_images, fmt="%d")
    print(len(wrong_images), wrong_images)

p_seed = int(input ("Seeds: ")) # input seeds value
p_trials = int(input ("Trials: ")) #input trial value
p_kernel_size = int (input ("Kernel size: ")) #input size kernel
p_logdir = input ("Logdir: ") #input model

for x in range (p_trials):
  run(p_seed + x, p_kernel_size, p_logdir)

Seeds: 0
Trials: 10
Kernel size: 5
Logdir: modelM5
108 [193, 359, 445, 449, 582, 625, 646, 659, 674, 716, 938, 947, 956, 965, 1014, 1156, 1226, 1232, 1290, 1299, 1328, 1527, 1621, 1681, 1709, 1790, 1868, 1901, 2035, 2040, 2098, 2118, 2135, 2182, 2189, 2224, 2266, 2293, 2308, 2358, 2462, 2488, 2582, 2597, 2823, 2927, 2939, 3005, 3073, 3132, 3225, 3384, 3422, 3475, 3534, 3558, 3762, 3767, 3808, 3821, 3846, 3853, 3869, 3906, 3985, 4065, 4176, 4201, 4248, 4284, 4369, 4443, 4497, 4547, 4551, 4571, 4740, 4759, 4761, 4823, 4874, 4990, 5165, 5457, 5888, 5955, 6065, 6505, 6558, 6576, 7233, 8020, 8094, 8279, 8408, 8504, 8527, 9009, 9015, 9530, 9627, 9664, 9679, 9693, 9729, 9749, 9754, 9904]
100 [8, 193, 358, 445, 449, 571, 716, 720, 726, 740, 938, 1039, 1112, 1156, 1226, 1232, 1242, 1299, 1328, 1393, 1403, 1522, 1721, 1737, 1754, 1847, 1868, 1901, 1911, 2024, 2035, 2040, 2098, 2118, 2130, 2148, 2182, 2224, 2293, 2433, 2447, 2462, 2488, 2597, 2823, 3073, 3132, 3225, 3384, 3422, 3534, 3558, 3726, 

## homo ensembel

In [ ]:
import numpy as np 

cnt = 1
best = 10000
curr = 10000

p_kernel_size = int (input ("Kernel size: ")) #input size kernel
KERNEL_SIZE = p_kernel_size

for i in range(10):
    for j in range(i+1,10):
        for k in range(j+1,10):
            w1 = np.loadtxt("MnistSimpleCNN/logs/modelM%d/wrong%03d.txt"%(KERNEL_SIZE, i)).astype(np.int)
            w2 = np.loadtxt("MnistSimpleCNN/logs/modelM%d/wrong%03d.txt"%(KERNEL_SIZE, j)).astype(np.int)
            w3 = np.loadtxt("MnistSimpleCNN/logs/modelM%d/wrong%03d.txt"%(KERNEL_SIZE, k)).astype(np.int)

            board = np.zeros((10000))
            board[w1] += 1
            board[w2] += 1
            board[w3] += 1
            board = board // 2
            curr = np.sum(board)
            if curr < best:
                best = curr
            print("%4d %4d %4d %4d %4d %4d"%(cnt, len(w1), len(w2), len(w3), curr, best))
            cnt += 1

Kernel size: 5
   1  108  100   76   76   76
   2  108  100   88   86   76
   3  108  100  106   88   76
   4  108  100   73   79   76
   5  108  100   99   85   76
   6  108  100  102   85   76
   7  108  100   83   77   76
   8  108  100  104   93   76
   9  108   76   88   73   73
  10  108   76  106   79   73
  11  108   76   73   66   66
  12  108   76   99   80   66
  13  108   76  102   80   66
  14  108   76   83   71   66
  15  108   76  104   86   66
  16  108   88  106   82   66
  17  108   88   73   75   66
  18  108   88   99   80   66
  19  108   88  102   85   66
  20  108   88   83   77   66
  21  108   88  104   86   66
  22  108  106   73   74   66
  23  108  106   99   91   66
  24  108  106  102   90   66
  25  108  106   83   84   66
  26  108  106  104   90   66
  27  108   73   99   75   66
  28  108   73  102   74   66
  29  108   73   83   71   66
  30  108   73  104   82   66
  31  108   99  102   82   66
  32  108   99   83   83   66
  33  108   99  104   92 